In [1]:
import numpy as np;
import pandas as pd;


## Initialize Parameters

In [2]:
## parameters and global variables

district = "all"

zetaPanchayat = 0.1
zetaMunicipality = 0.2
zetaCorporation = 0.3


muForSmall = 0.09
muForMedium = 0.04
muForLarge = 0.02



## Load and Initialize Data

In [3]:
## Load population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")

## Load distance data
distanceData = pd.read_csv("../data/" + district + "_distance_matrix.csv")

## Load area data
areaData = pd.read_csv("../data/" + district + "_area_data.csv")

## Number of regions
r = len(initDataDF.index)

## Sort population data and distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)
distanceData.sort_values(by=['fromName', 'toName'], inplace=True)
areaData.sort_values(by=['name'], inplace=True)


## Initialize Job 
initDataDF.loc[initDataDF['type']=='P','J'] = initDataDF[initDataDF['type']=='P']['N']*zetaPanchayat
initDataDF.loc[initDataDF['type']=='M','J'] = initDataDF[initDataDF['type']=='M']['N']*zetaMunicipality
initDataDF.loc[initDataDF['type']=='C','J'] = initDataDF[initDataDF['type']=='C']['N']*zetaCorporation

## Initialize T
areaData['areaSqKm'] = areaData['area']/(1000*1000)

initDataDF.loc[areaData['areaSqKm'] <= 25,'T'] = initDataDF[areaData['areaSqKm'] <= 25]['N']*muForSmall
initDataDF.loc[(areaData['areaSqKm'] > 25) & (areaData['areaSqKm'] <= 100),'T'] = initDataDF[(areaData['areaSqKm'] > 25) & (areaData['areaSqKm'] <= 100)]['N']*muForMedium
initDataDF.loc[areaData['areaSqKm'] > 100,'T'] = initDataDF[areaData['areaSqKm'] > 100]['N']*muForLarge

initDataDF.tail(15)

/Users/sreeramh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/sreeramh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/sreeramh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,name,type,S,E,I,H,R,N,J,T
1021,Vettikkavala__Kollam,P,36204,0,0,0,0,36204,3620.4,1448.16
1022,Vettom__Malappuram,P,6577,0,0,0,0,6577,657.7,263.08
1023,Vettoor__Thiruvananthapuram,P,18704,0,0,0,0,18704,1870.4,748.16
1024,Vijayapuram__Kottayam,P,30838,0,0,0,0,30838,3083.8,1233.52
1025,Vilakudi__Kollam,P,32995,0,0,0,0,32995,3299.5,2969.55
1026,Vilappil__Thiruvananthapuram,P,36212,0,0,0,0,36212,3621.2,3259.08
1027,Vilavoorkal__Thiruvananthapuram,P,31761,0,0,0,0,31761,3176.1,1270.44
1028,Vilayoor__Palakkad,P,23389,0,0,0,0,23389,2338.9,2105.01
1029,Villiyappally__Kozhikode,P,34502,0,0,0,0,34502,3450.2,1380.08
1030,Vithura__Thiruvananthapuram,P,26249,0,0,0,0,26249,2624.9,2362.41


## Matrix computation of TMatrix

### Prepare Distance matrix

In [4]:
distanceMatrix = distanceData['value'].to_numpy().reshape(r,r)
np.fill_diagonal(distanceMatrix,1)
distanceMatrix


array([[     1,  62713, 257069, ...,  38737, 153934, 239018],
       [ 62713,      1, 194549, ...,  74778, 216606, 176575],
       [257069, 194549,      1, ..., 263797, 410410,  18277],
       ...,
       [ 38737,  74778, 263797, ...,      1, 159119, 246386],
       [153934, 216606, 410410, ..., 159119,      1, 392249],
       [239018, 176575,  18277, ..., 246386, 392249,      1]])

### Job matrix

In [5]:
jobMatrix = np.tile(initDataDF['J'].to_numpy(),(r,1))
jobMatrix


array([[ 733.8, 3197.3, 3363.8, ..., 1023.9, 2931.6, 1817.6],
       [ 733.8, 3197.3, 3363.8, ..., 1023.9, 2931.6, 1817.6],
       [ 733.8, 3197.3, 3363.8, ..., 1023.9, 2931.6, 1817.6],
       ...,
       [ 733.8, 3197.3, 3363.8, ..., 1023.9, 2931.6, 1817.6],
       [ 733.8, 3197.3, 3363.8, ..., 1023.9, 2931.6, 1817.6],
       [ 733.8, 3197.3, 3363.8, ..., 1023.9, 2931.6, 1817.6]])

### Job by Distance Square matrix

In [6]:
tPropJByDSqMatrix = jobMatrix/np.multiply(distanceMatrix,distanceMatrix)
np.fill_diagonal(tPropJByDSqMatrix,0)
tPropJByDSqMatrix


/Users/sreeramh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[0.00000000e+00, 8.12958235e-07, 5.09015198e-08, ...,
        6.82347455e-07, 1.23718773e-07, 3.18153788e-08],
       [1.86578911e-07, 0.00000000e+00, 8.88734748e-08, ...,
        1.83109068e-07, 6.24832698e-08, 5.82961514e-08],
       [1.11039703e-08, 8.44744518e-08, 0.00000000e+00, ...,
        1.47135625e-08, 1.74047923e-08, 5.44112233e-06],
       ...,
       [4.89019008e-07, 5.71788869e-07, 4.83381986e-08, ...,
        0.00000000e+00, 1.15787221e-07, 2.99409972e-08],
       [3.09676748e-08, 6.81463223e-08, 1.99707464e-08, ...,
        4.04402155e-08, 0.00000000e+00, 1.18133922e-08],
       [1.28444789e-08, 1.02547472e-07, 1.00697884e-05, ...,
        1.68665201e-08, 1.90537746e-08, 0.00000000e+00]])

### Sum of Job by Distance Square matrix

In [7]:
tPropMatrixSum = np.tile(tPropJByDSqMatrix.sum(axis=1),(r,1)).T
tPropMatrixSum

array([[0.00263173, 0.00263173, 0.00263173, ..., 0.00263173, 0.00263173,
        0.00263173],
       [0.00343086, 0.00343086, 0.00343086, ..., 0.00343086, 0.00343086,
        0.00343086],
       [0.0036829 , 0.0036829 , 0.0036829 , ..., 0.0036829 , 0.0036829 ,
        0.0036829 ],
       ...,
       [0.000885  , 0.000885  , 0.000885  , ..., 0.000885  , 0.000885  ,
        0.000885  ],
       [0.0006826 , 0.0006826 , 0.0006826 , ..., 0.0006826 , 0.0006826 ,
        0.0006826 ],
       [0.00333536, 0.00333536, 0.00333536, ..., 0.00333536, 0.00333536,
        0.00333536]])

### Ti propotion matrix

In [8]:
tPropMatrix = tPropJByDSqMatrix/tPropMatrixSum
tPropMatrix

/Users/sreeramh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[0.00000000e+00, 3.08906795e-04, 1.93414922e-05, ...,
        2.59277483e-04, 4.70104956e-05, 1.20891655e-05],
       [5.43824846e-05, 0.00000000e+00, 2.59041086e-05, ...,
        5.33711233e-05, 1.82121090e-05, 1.69916822e-05],
       [3.01500510e-06, 2.29369222e-05, 0.00000000e+00, ...,
        3.99509948e-06, 4.72583554e-06, 1.47740053e-03],
       ...,
       [5.52564225e-04, 6.46089555e-04, 5.46194704e-05, ...,
        0.00000000e+00, 1.30833107e-04, 3.38316581e-05],
       [4.53673961e-05, 9.98338175e-05, 2.92569839e-05, ...,
        5.92445925e-05, 0.00000000e+00, 1.73065252e-05],
       [3.85100075e-06, 3.07455361e-05, 3.01909970e-03, ...,
        5.05687946e-06, 5.71265682e-06, 0.00000000e+00]])

### Ti matrix 
`number of people traveling out of region-i`

In [9]:
TiMatrix = np.tile(initDataDF['T'].to_numpy(),(r,1)).T
TiMatrix

array([[ 293.52,  293.52,  293.52, ...,  293.52,  293.52,  293.52],
       [2877.57, 2877.57, 2877.57, ..., 2877.57, 2877.57, 2877.57],
       [3027.42, 3027.42, 3027.42, ..., 3027.42, 3027.42, 3027.42],
       ...,
       [ 921.51,  921.51,  921.51, ...,  921.51,  921.51,  921.51],
       [2638.44, 2638.44, 2638.44, ..., 2638.44, 2638.44, 2638.44],
       [1635.84, 1635.84, 1635.84, ..., 1635.84, 1635.84, 1635.84]])

### Tij Matrix

In [10]:
TijMatrix = np.multiply(TiMatrix, tPropMatrix)
TijMatrix

array([[0.00000000e+00, 9.06703223e-02, 5.67711478e-03, ...,
        7.61031268e-02, 1.37985207e-02, 3.54841187e-03],
       [1.56489406e-01, 0.00000000e+00, 7.45408858e-02, ...,
        1.53579143e-01, 5.24066184e-02, 4.88947548e-02],
       [9.12768673e-03, 6.94396970e-02, 0.00000000e+00, ...,
        1.20948441e-02, 1.43070890e-02, 4.47271191e+00],
       ...,
       [5.09193459e-01, 5.95377986e-01, 5.03323882e-02, ...,
        0.00000000e+00, 1.20564016e-01, 3.11762113e-02],
       [1.19699153e-01, 2.63405537e-01, 7.71927967e-02, ...,
        1.56313303e-01, 0.00000000e+00, 4.56622284e-02],
       [6.29962107e-03, 5.02947778e-02, 4.93876405e+00, ...,
        8.27224569e-03, 9.34499253e-03, 0.00000000e+00]])

### Add Ni - Tij to diagonal elements

In [11]:
row,col = np.diag_indices_from(TijMatrix)
TijMatrix[row,col] = initDataDF['N'].to_numpy() - TijMatrix.sum(axis=1)
TijMatrix

array([[7.04448000e+03, 9.06703223e-02, 5.67711478e-03, ...,
        7.61031268e-02, 1.37985207e-02, 3.54841187e-03],
       [1.56489406e-01, 2.90954300e+04, 7.45408858e-02, ...,
        1.53579143e-01, 5.24066184e-02, 4.88947548e-02],
       [9.12768673e-03, 6.94396970e-02, 3.06105800e+04, ...,
        1.20948441e-02, 1.43070890e-02, 4.47271191e+00],
       ...,
       [5.09193459e-01, 5.95377986e-01, 5.03323882e-02, ...,
        9.31749000e+03, 1.20564016e-01, 3.11762113e-02],
       [1.19699153e-01, 2.63405537e-01, 7.71927967e-02, ...,
        1.56313303e-01, 2.66775600e+04, 4.56622284e-02],
       [6.29962107e-03, 5.02947778e-02, 4.93876405e+00, ...,
        8.27224569e-03, 9.34499253e-03, 1.65401600e+04]])

## Save output

In [12]:

data = []

for i in range(0,r):
    for j in range(0,r):
        Tij = TijMatrix[i][j]
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);

resultsDF.to_csv("../output/" + district + "_tmatrix_results.csv")


## Save top 5 

In [13]:
resultsDF["rank"] = resultsDF.groupby(['iName'])['Tij'].rank("dense", ascending=False)
top5resultsDF = resultsDF[resultsDF["rank"] <= 6]
top5resultsDF["jName_Tij"] = top5resultsDF["jName"] + " (" + top5resultsDF["Tij"].astype(str)  + ")"
top5resultsDFPivot = top5resultsDF.pivot(index="iName", columns="rank", values="jName_Tij")
top5resultsDFPivot.to_csv("../output/" + district + "_tmatrix_top5_regions.csv")


/Users/sreeramh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Old Version
### Function definition

def getTij(i,j,distanceMatrix, initDataDF, tPropMatrix):
    Ti = initDataDF.iloc[i]['T']

    if(i==j):
        return 0;
    
    Tjprop = tPropMatrix[i][j]
    TjpropSum = tPropMatrix[i].sum();
    ## TODO move this tPropMatrix computation.
    return Ti*Tjprop/TjpropSum;
    
    

### Compute TMatrix


data = []

for i in range(0,r):
    TijSum = 0
    for j in range(0,r):
        Tij = getTij(i,j,distanceMatrix, initDataDF, tPropMatrix)
        TijSum += Tij
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);

